## Spam Email Classifier with KNN using TF-IDF scores

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   The report file must be a well documented jupyter notebook, explaining the experiments you have performed, evaluation metrics and corresponding code. The code must run and be able to reproduce the accuracies, figures/graphs etc.
5.   For all the questions, you must create a train-validation data split and test the hyperparameter tuning on the validation set. Your jupyter notebook must reflect the same.
6.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

**Task: Given an email, classify it as spam or ham**

Given input text file ("emails.txt") containing 5572 email messages, with each row having its corresponding label (spam/ham) attached to it.

This task also requires basic pre-processing of text (like removing stopwords, stemming/lemmatizing, replacing email_address with 'email-tag', etc..).

You are required to find the tf-idf scores for the given data and use them to perform KNN using Cosine Similarity.

### Import necessary libraries

In [101]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
import math
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load dataset

In [102]:
df=pd.read_csv('emails.txt',sep='\t',names=["category","email"])
for i in range(4500, 5572):
    df = df.drop([i])

df

,category,email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
4495,ham,Man this bus is so so so slow. I think you're ...
4496,ham,Hope this text meets you smiling. If not then ...
4497,ham,"In case you wake up wondering where I am, I fo..."
4498,ham,Ok


### Preprocess data

Remove Whitespace function



In [103]:
def remove_whitespace(txt):
  txt.strip()
  return txt

Remove Stopwords


In [104]:
en_stopwords = stopwords.words('english')
def remove_stopwords(txt):
    ans=[]
    for token in txt:
        if token in en_stopwords or token in string.digits:
            continue
        else:
            ans.append(token)
    return ans

###Remove punctuation

In [105]:
def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

In [106]:
def stemming(text):
  porter = PorterStemmer()
  ans=list()
  for word in text:
      ans.append(porter.stem(word))
  return ans


In [107]:
df['email']=df['email'].str.lower()
df['email']=df['email'].apply(lambda x: remove_whitespace(x))
df['email']=df['email'].apply(lambda X: word_tokenize(X))
df['email'] = df['email'].apply(lambda x: remove_stopwords(x))
# df

In [108]:
df['email'] = df['email'].apply(lambda x: remove_punct(x))
# df

In [109]:
df['email']=df['email'].apply(lambda x: stemming(x))
# df

### Split data

In [110]:
training_data, validate_data, testing_data = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])
training_data.reset_index(inplace = True, drop = True)
testing_data.reset_index(inplace = True, drop = True)
validate_data.reset_index(inplace = True, drop = True)
for i in range(len(testing_data)):
  if(testing_data['email'][i]==[]):
    testing_data.drop(i,inplace=True)
for i in range(len(training_data)):
  if(training_data['email'][i]==[]):
    training_data.drop(i,inplace=True)
for i in range(len(validate_data)):
  if(validate_data['email'][i]==[]):
    validate_data.drop(i,inplace=True)
training_data.reset_index(inplace = True, drop = True)
testing_data.reset_index(inplace = True, drop = True)
validate_data.reset_index(inplace = True, drop = True)
training_data

,category,email
0,ham,"[stalk, u]"
1,ham,"[wake, lt, gt, morn]"
2,ham,"[happi, new, year, princess]"
3,spam,"[hot, live, fantasi, call, 08707509020, 20p, p..."
4,ham,"[go, ride, bike]"
...,...,...
2693,ham,"[could, seen, me, i, did, t, recognis, face]"
2694,ham,"[messag, truro, hospit, ext, phone, phone, side]"
2695,ham,"[liter, bed, like, lt, gt, hour]"
2696,spam,"[80488, 500, free, text, messag, valid, 31, de..."


### Train your KNN model (reuse previously iplemented model built from scratch) and test on your data

***1. Experiment with different distance measures [Euclidean distance, Manhattan distance, Hamming Distance] and compare with the Cosine Similarity distance results.***

### TF IDF ON TRAINING AND VALIDATE


In [111]:
def find_idf(sample_data):
  frequency_counter={}
  for list1 in sample_data['email']:
    set_words=set()
    for word in list1:
      set_words.add(word)
    for word in set_words:
      if word in frequency_counter:
        frequency_counter[word]=frequency_counter[word]+1
      else:
        frequency_counter[word]=1
  idf_score={}
  no_of_sentences=len(sample_data['email'])
  for word in frequency_counter:
    idf_score[word]=math.log(float(no_of_sentences)/frequency_counter[word])
  
  return idf_score

def find_tf(sample_data,idf_score,word_to_index):
    tf_idf=list()
    for txt in sample_data['email']:
      if txt==[]:
        continue
      tf_sent=[0 for x in range(len(idf_score))]
      for word in txt:
        if word not in idf_score:
          continue
        else:
          tf_sent[word_to_index[word]]=tf_sent[word_to_index[word]]+1
      for word in idf_score:
        tf_sent[word_to_index[word]]=(tf_sent[word_to_index[word]]/float(len(txt)))*idf_score[word]
      tf_idf.append(tf_sent)
    return tf_idf

idf_score=find_idf(training_data)
word_to_index={}
index=0
for word in idf_score:
   word_to_index[word]=index
   index=index+1
training_data_tf_idf=find_tf(training_data,idf_score,word_to_index)
validate_data_tf_idf=find_tf(validate_data,idf_score,word_to_index)
training_data_tf_idf

Output hidden; open in https://colab.research.google.com to view.

### DISTANCE FUNCTION

In [112]:
def find_cosine(validate_data_tf_idf,training_data_tf_idf):
  cosine_list=[]
  for i1 in range(len(validate_data_tf_idf)):
      temp_list=[]
      list1=np.array(validate_data_tf_idf[i1])
      norm_list1=norm(list1)
      for i2 in range(len(training_data_tf_idf)):
        list2=np.array(training_data_tf_idf[i2])
        norm_list2=norm(list2)
        cosine_dist=dot(list1, list2)/(norm_list1*norm_list2)
        l1=[i1,i2,cosine_dist]
        temp_list.append(l1)
      temp_list.sort(key=lambda x: x[2],reverse=True)
      cosine_list.append(temp_list)
  return cosine_list

def find_eucledian(validate_data_tf_idf,training_data_tf_idf):
  eucledian_list=[]
  for i1 in range(len(validate_data_tf_idf)):
      temp_list=[]
      list1=np.array(validate_data_tf_idf[i1])
      for i2 in range(len(training_data_tf_idf)):
        list2=np.array(training_data_tf_idf[i2])
        euc_dist=np.sqrt(np.sum(np.square(list1 - list2)))
        l1=[i1,i2,euc_dist]
        temp_list.append(l1)
      temp_list.sort(key=lambda x: x[2])
      eucledian_list.append(temp_list)
  return eucledian_list

def find_manhattan(validate_data_tf_idf,training_data_tf_idf): 
  manhattan_list=[]
  for i1 in range(len(validate_data_tf_idf)):
      temp_list=[]
      list1=np.array(validate_data_tf_idf[i1])
      for i2 in range(len(training_data_tf_idf)):
        list2=np.array(training_data_tf_idf[i2])
        man_dist=np.abs(list1 - list2).sum()
        l1=[i1,i2,man_dist]
        temp_list.append(l1)
      temp_list.sort(key=lambda x: x[2])
      manhattan_list.append(temp_list)
  print(len(manhattan_list))
  return manhattan_list


### F1 Score and KNN

In [113]:
def find_knn(k,distance_list):
    predicted_list=[]
    for list1 in distance_list:
      ham=0
      spam=0
      for i in range(k):
        if(training_data['category'][list1[i][1]]=="ham"):
          ham=ham+1
        else:
          spam=spam+1
      if(ham>spam):
         predicted_list.append("ham")
      else:
         predicted_list.append("spam")
    return predicted_list

def find_f1_score(answer_list,predicted_list):
  true_positive=0
  true_negative=0
  false_positive=0
  false_negative=0
  for i in range(len(predicted_list)):
    if(answer_list[i]=="spam"):
      if(answer_list[i]==predicted_list[i]):
        true_positive+=1
      else:
        false_negative+=1
    else:
      if(answer_list[i]==predicted_list[i]):
        true_negative+=1
      else:
        false_positive+=1
  print("{} {} {} {}".format(true_positive,true_negative,false_positive,false_negative))
  precision=0
  recall=0
  if((true_positive+false_positive)>0):
    precision=float(true_positive)/(true_positive+false_positive)
  if((true_positive+false_negative)>0):
    recall=float(true_positive)/(true_positive+false_negative)
  f1_score=0.0
  if((precision+recall)>0.0):
    f1_score=2.0*(float(precision*recall)/(precision+recall))
  return f1_score
    
def find_matrices(k,distance_list,validate_data):
  predicted_list=find_knn(k,distance_list)
  answer_list=[]
  for i in range(len(validate_data)):
    answer_list.append(validate_data['category'][i])
  return find_f1_score(answer_list,predicted_list)


***2. Explain which distance measure works best and why? Explore the distance measures and weigh their pro and cons in different application settings.***

***3. Report Mean Squared Error(MSE), Mean-Absolute-Error(MAE), R-squared (R2) score in a tabular form***

***4. Choose different K values (k=1,3,5,7,11,17,23,28) and experiment. Plot a graph showing R2 score vs k.***

In [ ]:
import matplotlib.pyplot as plt
k=[1,3,5,7,11,17,23,28]
cosine_distance_list=find_cosine(validate_data_tf_idf,training_data_tf_idf)
euc_distance_list=find_eucledian(validate_data_tf_idf,training_data_tf_idf)
man_distance_list=find_manhattan(validate_data_tf_idf,training_data_tf_idf)
f1_score_cosine=list()
f1_score_euc=list()
f1_score_man=list()
for i in k:
  f1_score_cosine.append(find_matrices(i,cosine_distance_list,validate_data))
  print("k is {} done for cosine".format(i))
  f1_score_euc.append(find_matrices(i,euc_distance_list,validate_data))
  print("k is {} done for eucledian".format(i))
  f1_score_man.append(find_matrices(i,man_distance_list,validate_data))
  print("k is {} done for manhattan".format(i))
print(f1_score_cosine)
print(f1_score_euc)
print(f1_score_man)


f1_score_cosine_temp=f1_score_cosine[1:]
f1_score_euc_temp=f1_score_euc[1:]
f1_score_man_temp=f1_score_man[1:]

max_cosine = f1_score_cosine_temp.index(max(f1_score_cosine_temp))
max_euc = f1_score_euc_temp.index(max(f1_score_euc_temp))
max_man = f1_score_man_temp.index(max(f1_score_man_temp))
maxm_of_three=max(f1_score_cosine_temp[max_cosine],f1_score_euc_temp[max_euc],f1_score_man_temp[max_man])
optimal_k=k[max_cosine+1]
optimal_metric="cosine"
if(maxm_of_three==f1_score_euc_temp[max_euc]):
  optimal_k=k[max_euc+1]
  optimal_metric="eucledian"
elif(maxm_of_three==f1_score_man_temp[max_man]):
  optimal_k=k[max_man+1]
  optimal_metric="manhattan"
optimal_k
optimal_metric

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
test_data_tf_idf=find_tf(testing_data,idf_score,word_to_index)
distance_list=list()
if(optimal_metric=="cosine"):
  distance_list=find_cosine(test_data_tf_idf,training_data_tf_idf)
elif(optimal_metric=="eucledian"):
  distance_list=find_eucledian(test_data_tf_idf,training_data_tf_idf)
else:
  distance_list=find_manhattan(test_data_tf_idf,training_data_tf_idf)
f1_score_test=find_matrices(optimal_k,distance_list,testing_data)
f1_score_test

### Train and test Sklearn's KNN classifier model on your data (use metric which gave best results on your experimentation with built-from-scratch model.)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
test_data_tf_idf=find_tf(testing_data,idf_score,word_to_index)
training_label=list()
test_label=list()
for label in training_data['category']:
  training_label.append(label)
for label in testing_data['category']:
  test_label.append(label)
neigh = KNeighborsClassifier(n_neighbors=optimal_k,metric="cosine")
neigh.fit(training_data_tf_idf, training_label)
pred=neigh.predict(test_data_tf_idf)
f1_score_skl=f1_score(test_label,pred,pos_label='spam')
f1_score_skl


***Compare both the models result.***

***What is the time complexity of training using KNN classifier?***

***What is the time complexity while testing? Is KNN a linear classifier or can it learn any boundary?***